# Parser Prototype

In [ ]:
from lark import Lark

with open("grammar.lark") as grammar_file:
    grammar = grammar_file.read()

In [ ]:
# vm, iface = get_consts(smtsorts, ["vm", "iface"])
# return And(
#     smtenc.element_class_fun(vm) == smtenc.classes["infrastructure_VirtualMachine"],
#     Not(
#         Exists(
#             [iface],
#             ENCODINGS.association_rel(vm, smtenc.associations["infrastructure_ComputingNode::ifaces"], iface)
#         )
#     )
# )

expr_to_parse = r"""
    # Expr to parse
    not vm is infrastructure.VirtualMachine
    and
    vm is not infrastructure.Storage
    iff
    not exists iface, apple (
        (vm has association infrastructure.ComputingNode->ifaces iface)
        or
        (vm has attribute infrastructure.ComputingNode->os Os1)
    )
"""

In [ ]:
parser = Lark(grammar, start="start")
tree = parser.parse(expr_to_parse)

print(tree.pretty())

We need the `ModelChecker` to import `SMTEncodings` and `SMTSorts` in order to create our Z3 constants programmatically.

Now the model checker should expose the *intermediate model checker* which should provide us with those two collections.

In [ ]:
from mc_openapi.doml_mc import ModelChecker, DOMLVersion
from mc_openapi.doml_mc.imc import IntermediateModelChecker

# Import DOMLX as bytes
doml_document_path = "../../tests/doml/faas.domlx"
with open(doml_document_path, "rb") as xmif:
    doml_xmi = xmif.read()

model_checker = ModelChecker(doml_xmi, DOMLVersion.V2_0)

intermediate_model_checker = IntermediateModelChecker(
    model_checker.metamodel,
    model_checker.inv_assoc,
    model_checker.intermediate_model
)

ENCODINGS =  intermediate_model_checker.smt_encoding
SORTS = intermediate_model_checker.smt_sorts

assert ENCODINGS and SORTS

I think we don't need to have the user declare each constant and value explicitly.
We can take care of that by saving the first occurence of each in a dictionary,
and subsequent uses of a variable will fetch the corresponding z3 ref from the dictionary.

In [ ]:
from mc_openapi.dsl_parser.utils import UniqueVarStore

constants_store = UniqueVarStore(ENCODINGS, SORTS.element_sort)
values_store = UniqueVarStore(ENCODINGS, SORTS.attr_data_sort)

RefHandler provides methods to create associations, classes, etc. to the Transformer without exposing Encodings and Sorts to it.

In [ ]:
from mc_openapi.dsl_parser.utils import RefHandler

ref_handler = RefHandler(ENCODINGS)

# Test existent key
vm = ref_handler.get_class("infrastructure.VirtualMachine")
print("type: \t", type(vm))
print("value: \t", vm)
try:
    bad_class = ref_handler.get_class("infrastructure.Bicycle")
except Exception: 
    print("Exception catched!")

The parser will now produce a Z3 expression to evaluate.

In [ ]:
from z3 import Not, And, Or, Xor, Implies, Exists, ForAll

from mc_openapi.dsl_parser.parser import Parser

parser = Parser(grammar, ENCODINGS, SORTS)

z3_parsed_exp, z3_free_vars = parser.parse(expr_to_parse)

print(z3_parsed_exp)
print("TYPE:", type(z3_parsed_exp))
print("FREE VARS:", z3_free_vars)